In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt

In [2]:
df = pd.read_csv('dataset/AGE_PREDICTION.csv')
df.head()

,feat_1,feat_2,feat_3,feat_4,feat_5,feat_6,feat_7,feat_8,feat_9,feat_10,...,feat_24,feat_25,feat_26,feat_27,feat_28,feat_29,feat_30,feat_31,feat_32,gt
0,2.686191,-0.989465,-0.920503,1.607427,-0.896248,1.118974,-0.969456,1.811707,2.560955,3.803463,...,-0.862891,-0.909545,-0.915361,-0.952061,-0.989461,1.911855,1.409705,2.303997,-0.981840,54
1,-0.887917,4.915272,-0.939446,-0.343677,-0.964685,-0.478649,4.342395,-0.332870,-0.768041,-0.815375,...,-0.939201,-0.965917,-0.969461,-0.934799,5.304822,0.934790,-0.410701,0.284690,4.919212,18
2,-0.923215,2.746968,-0.918085,0.047804,-0.908587,-0.451752,2.984481,0.535007,-0.591029,-0.324043,...,-0.809726,-0.929934,-0.891814,-0.881796,3.415373,1.044108,-0.442615,0.033648,2.628199,26
3,-0.268866,-0.408416,-0.935145,0.731800,-0.922438,0.221781,-0.046606,1.149634,0.592136,1.357959,...,-0.834968,-0.937475,-0.917737,-0.929519,-0.226282,1.608048,0.276169,1.246468,-0.363367,33
4,0.529231,-0.829957,-0.897425,0.921280,-0.865304,0.331018,-0.644940,1.296097,1.166863,2.036034,...,-0.775411,-0.881967,-0.864018,-0.908001,-0.784495,1.329586,0.547925,1.195395,-0.810089,35


In [ ]:
class NeuralNetwork:
    def __init__(self, neurons, act='tanh'):
        """
        neurons: list with the number of neurons in each layer.
                      e.g. [512, 16, 8, 1]
        activation: string, 'tanh' or 'sigmoid'
        """
        self.neurons = neurons
        self.layers = len(neurons) - 1  # number of layers (excluding input)
        self.act = act

        # Initialize weights and biases
        self.weights = []
        self.biases = []
        for i in range(self.layers):
            in_dim = neurons[i]
            out_dim = neurons[i + 1]
            W = np.random.randn(in_dim, out_dim) * 0.01
            b = np.zeros((1, out_dim))
            self.weights.append(W)
            self.biases.append(b)

    def activation(self, x):
        if self.act == 'tanh':
            return np.tanh(x)
        elif self.act == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        else:
            raise ValueError("Unsupported activation function")
        
    def derivate_activation(self, x):
        if self.act == 'tanh':
            return 1-(self.activation(x)**2)
        elif self.act == 'sigmoid':
            return self.activation(x)*(1-self.activation(x))

    def forward(self, X):
        """
        Performs a forward pass through the network.
        X: input data of shape (n_samples, n_features)
        Returns:
            Output prediction of shape (n_samples, 1)
        """
        a = X
        zs = []
        activations = [X]  # Store input as the first activation
        
        for i in range(self.layers - 1):  # Hidden layers
            z = np.dot(a, self.weights[i]) + self.biases[i]
            a = self.activation(z)
            zs.append(z)
            activations.append(a)

        # Output layer (no activation)
        output = np.dot(a, self.weights[-1]) + self.biases[-1]
        zs.append(output)
        
        return output, activations, zs

    def get_params_vector(self):
        """Returns all weights and biases flattened into a single vector."""
        params = []
        for W, b in zip(self.weights, self.biases):
            params.append(W.flatten())
            params.append(b.flatten())
        return np.concatenate(params)
    

    def set_params_vector(self, flat_params):
        """Set the weights and biases from a flat parameter vector."""
        idx = 0
        self.weights = []
        self.biases = []
        for i in range(self.layers):
            in_dim = self.neurons[i]
            out_dim = self.neurons[i + 1]
            w_size = in_dim * out_dim
            b_size = out_dim

            W = flat_params[idx:idx + w_size].reshape((in_dim, out_dim))
            idx += w_size
            b = flat_params[idx:idx + b_size].reshape(1, out_dim)
            idx += b_size

            self.weights.append(W)
            self.biases.append(b)

    def mse_loss(self, y_real, y_pred, alpha):
        """
        y_real: true target values
        y_pred: predicted values
        alpha: regularization parameter
        """
        weights = self.weights

        loss = np.mean((y_real - y_pred)**2)/2
        reg = sum([np.sum(w**2) for w in weights])
        return loss + alpha * reg
    
    
    def mape(y_real, y_pred):
        """
        y_real: true target values
        y_pred: predicted values
        """
        return np.mean(np.abs((y_real - y_pred) / (y_real + 1e-8))) * 100
    
    def dloss_dypred(y_real, y_pred):
        return (y_pred-y_real)/y_real.shape[0]
    
    def backward(self, X, y, alpha = 0.5):
        y_pred, activations, zs = self.forward(X)
        deltas = [None] * self.layers
        grads_W, grads_b = [], []

        # delta output layer
        dL_dy = self.dloss_dypred(y, y_pred)  # shape (batch, out_dim)
        deltas[-1] = dL_dy  # ultimo layer: no activation

        # hidden layers backwards
        for layer in reversed(range(self.layers - 1)):
            da_dz = self.derivate_activation(zs[layer])
            deltas[layer] = (deltas[layer+1] @ self.weights[layer+1].T) * da_dz

        # grad W, b
        for layer in range(self.layers):
            a_prev = activations[layer]
            grad_W = (a_prev.T @ deltas[layer])+2*alpha*self.weights[layer]
            grads_W.append(grad_W)
            grads_b.append(np.sum(deltas[layer], axis=0))

        return grads_W, grads_b
    
    def update(self, grads_W, grads_b, lr=1e-3):
        for i in range(self.layers):
            self.weights[i] -= lr * grads_W[i]
            self.biases[i]  -= lr * grads_b[i]


    def train(model, X, y, epochs, lr, method="Batch", alpha = 0.5, batch_size=None, X_val = None, y_val = None):

        if method != "Batch" and method != "Mini Batch" and method != "SGD":
            raise ValueError('Select a method between: "Batch", "Mini Batch" and "SGD".')
        print(f"\nTraining started using '{method}' method for {epochs} epochs.")
        if method == "Mini Batch" and batch_size is None:
            if batch_size is None:
                raise ValueError("You must specify a batch_size for mini-batch training.")
            print(f"Mini-batch size: {batch_size}")

        for epoch in range(epochs):
            print(f"Epoch {epoch + 1}/{epochs}")

            if method == "Batch":
                # Use the entire dataset
                X_batch, y_batch = X, y

                loss = model.forward(X_batch, y_batch)
                model.backward(X_batch, y_batch, alpha=alpha)
                model.update(lr)

                print(f"[Batch] Loss: {loss:.6f}")

            elif method == "SGD":
                # Shuffle the data
                indices = np.random.permutation(len(X))
                X_shuffled = X[indices]
                y_shuffled = y[indices]

                total_loss = 0
                for i in range(len(X_shuffled)):
                    xi = X_shuffled[i].reshape(1, -1)
                    yi = y_shuffled[i].reshape(1, -1)

                    loss = model.forward(xi, yi)
                    model.backward(xi, yi, alpha=alpha)
                    model.update(lr)

                    total_loss += loss

                    print(f"[SGD] Sample {i+1}/{len(X_shuffled)} - Loss: {loss:.6f}")

                avg_loss = total_loss / len(X_shuffled)
                print(f"[SGD] Average Loss: {avg_loss:.6f}")

            elif method == "Mini Batch":
                # Shuffle the data
                indices = np.random.permutation(len(X))
                X_shuffled = X[indices]
                y_shuffled = y[indices]

                total_loss = 0
                num_batches = 0

                for i in range(0, len(X_shuffled), batch_size):
                    X_batch = X_shuffled[i:i+batch_size]
                    y_batch = y_shuffled[i:i+batch_size]

                    loss = model.forward(X_batch, y_batch)
                    model.backward(X_batch, y_batch, alpha=alpha)
                    model.update(lr)

                    total_loss += loss
                    num_batches += 1

                    print(f"[Mini-batch] Batch {num_batches} - Loss: {loss:.6f}")

                avg_loss = total_loss / num_batches
                print(f"[Mini-batch] Average Loss: {avg_loss:.6f}")
            
            if X_val is not None and y_val is not None:

                y_pred_val = model.forward(X_val, y_val)[0]

                val_loss = model.mse_loss(y_val, y_pred_val, alpha=alpha)

                print(f"Validation Loss: {val_loss:.4f}")

        print(f"Training completed!\nFinal training loss: ")

## Prova

In [13]:
class NeuralNetwork:
    def __init__(self, neurons, act='tanh'):
        """
        neurons: list with the number of neurons in each layer.
                      e.g. [512, 16, 8, 1]
        activation: string, 'tanh' or 'sigmoid'
        """
        self.neurons = neurons
        self.layers = len(neurons) - 1  # number of layers (excluding input)
        self.act = act

        # Initialize weights and biases
        self.weights = []
        self.biases = []
        for i in range(self.layers):
            in_dim = neurons[i]
            out_dim = neurons[i + 1]
            W = np.random.randn(in_dim, out_dim) * 0.01
            b = np.zeros((1, out_dim))
            self.weights.append(W)
            self.biases.append(b)

    def activation(self, x):
        if self.act == 'tanh':
            return np.tanh(x)
        elif self.act == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        else:
            raise ValueError("Unsupported activation function")
        
    def derivate_activation(self, x):
        if self.act == 'tanh':
            return 1-(self.activation(x)**2)
        elif self.act == 'sigmoid':
            return self.activation(x)*(1-self.activation(x))

    def forward(self, X):
        """
        Performs a forward pass through the network.
        X: input data of shape (n_samples, n_features)
        Returns:
            Output prediction of shape (n_samples, 1)
        """
        a = X
        zs = []
        activations = [X]  # Store input as the first activation
        
        for i in range(self.layers - 1):  # Hidden layers
            z = np.dot(a, self.weights[i]) + self.biases[i]
            a = self.activation(z)
            zs.append(z)
            activations.append(a)

        # Output layer (no activation)
        output = np.dot(a, self.weights[-1]) + self.biases[-1]
        zs.append(output)
        
        return output, activations, zs

    def get_params_vector(self):
        """Returns all weights and biases flattened into a single vector."""
        params = []
        for W, b in zip(self.weights, self.biases):
            params.append(W.flatten())
            params.append(b.flatten())
        return np.concatenate(params)
    

    def set_params_vector(self, flat_params):
        """Set the weights and biases from a flat parameter vector."""
        idx = 0
        self.weights = []
        self.biases = []
        for i in range(self.layers):
            in_dim = self.neurons[i]
            out_dim = self.neurons[i + 1]
            w_size = in_dim * out_dim
            b_size = out_dim

            W = flat_params[idx:idx + w_size].reshape((in_dim, out_dim))
            idx += w_size
            b = flat_params[idx:idx + b_size].reshape(1, out_dim)
            idx += b_size

            self.weights.append(W)
            self.biases.append(b)

    def mse_loss(self, y_real, y_pred, alpha):
        """
        y_real: true target values
        y_pred: predicted values
        alpha: regularization parameter
        """
        weights = self.weights

        loss = np.mean((y_real - y_pred)**2)/2
        reg = sum([np.sum(w**2) for w in weights])
        return loss + alpha * reg
    
    
    def mape(y_real, y_pred):
        """
        y_real: true target values
        y_pred: predicted values
        """
        return np.mean(np.abs((y_real - y_pred) / (y_real + 1e-8))) * 100
    
    def dloss_dypred(y_real, y_pred):
        return (y_pred-y_real)/y_real.shape[0]
    
    def backward(self, X, y, alpha = 0.5):
        y_pred, activations, zs = self.forward(X)
        deltas = [None] * self.layers
        grads_W, grads_b = [], []

        # delta output layer
        dL_dy = self.dloss_dypred(y, y_pred)  # shape (batch, out_dim)
        deltas[-1] = dL_dy  # ultimo layer: no activation

        # hidden layers backwards
        for layer in reversed(range(self.layers - 1)):
            da_dz = self.derivate_activation(zs[layer])
            deltas[layer] = (deltas[layer+1] @ self.weights[layer+1].T) * da_dz

        # grad W, b
        for layer in range(self.layers):
            a_prev = activations[layer]
            grad_W = (a_prev.T @ deltas[layer])+2*alpha*self.weights[layer]
            grads_W.append(grad_W)
            grads_b.append(np.sum(deltas[layer], axis=0))

        return grads_W, grads_b
    
    def update(self, grads_W, grads_b, lr=1e-3):
        for i in range(self.layers):
            self.weights[i] -= lr * grads_W[i]
            self.biases[i]  -= lr * grads_b[i]

    def train(self, X, y, epochs, lr, method="Batch", alpha=0.5, batch_size=None, X_val=None, y_val=None):
        if method not in ["Batch", "Mini Batch", "SGD"]:
            raise ValueError('Select a method between: "Batch", "Mini Batch", and "SGD".')

        print(f"\nTraining started using '{method}' method for {epochs} epochs.")

        if method == "Mini Batch" and batch_size is None:
            raise ValueError("You must specify a batch_size for mini-batch training.")

        for epoch in range(epochs):
            print(f"\nEpoch {epoch + 1}/{epochs}")

            if method == "Batch":
                X_batch, y_batch = X, y

                y_pred, _, _ = self.forward(X_batch)
                loss = self.mse_loss(y_batch, y_pred, alpha)

                grads_W, grads_b = self.backward(X_batch, y_batch, alpha)
                self.update(grads_W, grads_b, lr)

                print(f"[Batch] Loss: {loss:.6f}")

            elif method == "SGD":
                indices = np.random.permutation(len(X))
                X_shuffled, y_shuffled = X[indices], y[indices]

                total_loss = 0
                for i in range(len(X_shuffled)):
                    xi = X_shuffled[i].reshape(1, -1)
                    yi = y_shuffled[i].reshape(1, -1)

                    y_pred, _, _ = self.forward(xi)
                    loss = self.mse_loss(yi, y_pred, alpha)

                    grads_W, grads_b = self.backward(xi, yi, alpha)
                    self.update(grads_W, grads_b, lr)

                    total_loss += loss

                avg_loss = total_loss / len(X_shuffled)
                print(f"[SGD] Average Loss: {avg_loss:.6f}")

            elif method == "Mini Batch":
                indices = np.random.permutation(len(X))
                X_shuffled, y_shuffled = X[indices], y[indices]

                total_loss = 0
                num_batches = 0

                for i in range(0, len(X_shuffled), batch_size):
                    X_batch = X_shuffled[i:i+batch_size]
                    y_batch = y_shuffled[i:i+batch_size]

                    y_pred, _, _ = self.forward(X_batch)
                    loss = self.mse_loss(y_batch, y_pred, alpha)

                    grads_W, grads_b = self.backward(X_batch, y_batch, alpha)
                    self.update(grads_W, grads_b, lr)

                    total_loss += loss
                    num_batches += 1

                avg_loss = total_loss / num_batches
                print(f"[Mini-batch] Average Loss: {avg_loss:.6f}")

            # Validation
            if X_val is not None and y_val is not None:
                y_pred_val, _, _ = self.forward(X_val)
                val_loss = self.mse_loss(y_val, y_pred_val, alpha)
                print(f"Validation Loss: {val_loss:.6f}")

        print("Training completed!")


In [ ]:
import numpy as np
import pandas as pd

# Take a random subset of 1000 rows
df_subset = df.sample(n=1000, random_state=42)

# Split features and target
X = df_subset.drop("gt", axis=1).values
y = df_subset["gt"].values.reshape(-1, 1)

# Train/val split (80/20)
split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

# Define and train model
nn = NeuralNetwork([X.shape[1], 16, 8, 1], act="tanh")

nn.train(nn, 
      X_train, y_train, 
      lr=0.01, 
      epochs= 20,
      method="Mini Batch", 
      batch_size=32, 
      alpha=0.001, 
      X_val=X_val, y_val=y_val)



Training started using 'Mini Batch' method for [[38]
 [32]
 [55]
 [26]
 [34]
 [26]
 [24]
 [32]
 [25]
 [28]
 [27]
 [27]
 [34]
 [56]
 [40]
 [24]
 [35]
 [10]
 [25]
 [36]
 [47]
 [43]
 [36]
 [21]
 [30]
 [26]
 [45]
 [15]
 [32]
 [55]
 [66]
 [57]
 [28]
 [23]
 [20]
 [14]
 [39]
 [16]
 [35]
 [55]
 [26]
 [59]
 [24]
 [26]
 [25]
 [26]
 [26]
 [34]
 [27]
 [32]
 [17]
 [26]
 [28]
 [31]
 [27]
 [86]
 [30]
 [30]
 [26]
 [35]
 [53]
 [26]
 [65]
 [27]
 [12]
 [17]
 [47]
 [68]
 [39]
 [34]
 [36]
 [68]
 [23]
 [50]
 [16]
 [28]
 [28]
 [16]
 [27]
 [29]
 [31]
 [29]
 [16]
 [35]
 [30]
 [47]
 [26]
 [37]
 [45]
 [58]
 [24]
 [37]
 [48]
 [43]
 [25]
 [50]
 [35]
 [20]
 [38]
 [16]
 [36]
 [63]
 [41]
 [63]
 [32]
 [38]
 [53]
 [25]
 [52]
 [26]
 [85]
 [36]
 [65]
 [49]
 [26]
 [28]
 [85]
 [35]
 [41]
 [65]
 [21]
 [62]
 [36]
 [22]
 [30]
 [25]
 [10]
 [36]
 [26]
 [35]
 [30]
 [23]
 [26]
 [26]
 [60]
 [26]
 [35]
 [70]
 [25]
 [31]
 [36]
 [45]
 [23]
 [30]
 [24]
 [57]
 [64]
 [55]
 [66]
 [32]
 [55]
 [37]
 [31]
 [60]
 [79]
 [50]
 [32]
 [25]
 [38

TypeError: only integer scalar arrays can be converted to a scalar index